In [1]:
import os
import gc
import json
import time
import joblib
import warnings
import traceback
import numpy as np
import pandas as pd
import optuna
import pandas_ta as ta
import tensorflow as tf
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_ta as ta
from scipy.signal import argrelextrema
from numba import jit
from collections import Counter
import numpy as np
import pandas as pd
import optuna
import joblib
import json
import gc
import traceback
import os
from datetime import datetime
import tensorflow as tf
# Machine Learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, classification_report
)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler
# Numba (For Fast Backtesting)
from numba import jit

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# GPU Check
if tf.config.list_physical_devices('GPU'):
    print(f"🚀 GPU Detected: {tf.config.list_physical_devices('GPU')}")
else:
    print("⚠️ No GPU detected. Using CPU.")
    

def remove_raw_prices_and_transform(df, target_type, method):
    df_transformed = df.copy()
    
    # 1. 로그 수익률 생성
    if 'eth_log_return' not in df_transformed.columns:
        close_col = 'ETH_Close' if 'ETH_Close' in df_transformed.columns else 'close'
        if close_col in df_transformed.columns:
            df_transformed['eth_log_return'] = np.log(df_transformed[close_col] / df_transformed[close_col].shift(1))
    
    # 2. ATR 변환 (달러 -> 비율)
    if 'ATR_84' in df_transformed.columns:
        close_col = 'ETH_Close' if 'ETH_Close' in df_transformed.columns else 'close'
        if close_col in df_transformed.columns:
            df_transformed['ATR_84'] = (df_transformed['ATR_84'] / df_transformed[close_col]) * 100
            df_transformed['ATR_84'] = df_transformed['ATR_84'].replace([np.inf, -np.inf], 0).fillna(0)

    remove_patterns = [
        '_Close', '_Open', '_High', '_Low', '_Volume', 
        'tvl', 'mcap', 'sp500', 'gold', 'dxy', 'vwap' 
    ]
    
    keep_keywords = [
        '_pct', '_ratio', '_chg', '_diff', '_dist', '_score', 'zscore', 
        'natr', 'rsi', 'mfi', 'adx', 'cci', 'vr', 'shadow', 
        'div', 'spike', 'climax', 'absorption', 'breakout', 'price_vs',
        'ATR_84'
    ]
    
    cols_to_remove = []
    for col in df_transformed.columns:
        if col in ['date', 'target', 'next_direction', 'next_close', 'next_log_return', 'take_profit_price', 'stop_loss_price', 'real_entry_price', 'ATR_84']:
            continue
            
        is_dangerous = any(p in col.lower() for p in remove_patterns) or col in ['EMA_72', 'SMA_120', 'OBV']
        is_safe = any(k in col.lower() for k in keep_keywords)
        
        if is_dangerous and not is_safe:
            cols_to_remove.append(col)

    if cols_to_remove:
        df_transformed.drop(cols_to_remove, axis=1, inplace=True)
        
    return df_transformed



2025-11-29 19:16:59.987238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-29 19:16:59.987279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-29 19:16:59.988665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-29 19:16:59.995798: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 19:17:00.757098: W tensorflow/comp

🚀 GPU Detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
@jit(nopython=True)
def compute_targets_with_hourly_numba_4h(
    base_dates_ts, base_atr, base_adx, 
    hourly_dates_ts, hourly_open, hourly_high, hourly_low, 
    lookahead_periods, profit_mult, stop_mult,
    use_dynamic_tp, adx_th_high, adx_th_low, boost_factor, penalty_factor 
):
    n = len(base_dates_ts)
    targets = np.full(n, -1, dtype=np.int32)
    entry_prices = np.full(n, np.nan, dtype=np.float64)
    tp_prices = np.full(n, np.nan, dtype=np.float64)
    sl_prices = np.full(n, np.nan, dtype=np.float64)
    
    four_hour_ms = 14400000
    lookahead_ms = lookahead_periods * four_hour_ms
    
    h_start = 0
    
    for i in range(n - 1):
        atr = base_atr[i]
        adx = base_adx[i] 
        
        if np.isnan(atr) or atr <= 0: continue
        
        current_profit_mult = profit_mult
        
        if use_dynamic_tp:
            if adx >= adx_th_high: 
                current_profit_mult *= boost_factor
            elif adx <= adx_th_low:
                current_profit_mult *= penalty_factor
        
        entry_start_ts = base_dates_ts[i] + four_hour_ms
        entry_end_ts = entry_start_ts + lookahead_ms
        
        first_entry_idx = -1
        
        for h in range(h_start, len(hourly_dates_ts)):
            if hourly_dates_ts[h] >= entry_start_ts:
                first_entry_idx = h
                h_start = h 
                break
        
        if first_entry_idx == -1: continue
        
        entry_price = hourly_open[first_entry_idx]
        
        tp = entry_price + (atr * current_profit_mult)
        sl = entry_price - (atr * stop_mult)
        
        entry_prices[i] = entry_price
        tp_prices[i] = tp
        sl_prices[i] = sl
        
        result = 0 
        final_idx = -1 
        
        for h in range(first_entry_idx, len(hourly_dates_ts)):
            if hourly_dates_ts[h] >= entry_end_ts:
                final_idx = h
                break
            if hourly_low[h] <= sl:
                result = 0
                final_idx = -1 
                break
            if hourly_high[h] >= tp:
                result = 1
                final_idx = -1 
                break
        
        if final_idx != -1:
            exit_price = hourly_open[final_idx]
            if (exit_price - entry_price) >= atr:
                result = 1
            else:
                result = 0

        targets[i] = result
    
    return targets, entry_prices, tp_prices, sl_prices


def create_targets_4h(df_4h, df_1h, lookahead=30, profit_mult=1.5, stop_mult=1.0, **kwargs):
    df_target = df_4h.copy()
    hourly_df = df_1h.copy()
    
    use_dynamic_tp = kwargs.get('use_dynamic_tp', False)
    adx_th_high = kwargs.get('adx_threshold', 40) 
    adx_th_low = kwargs.get('adx_threshold_low', 20)
    boost_factor = kwargs.get('boost_factor', 1.6) 
    penalty_factor = kwargs.get('penalty_factor', 0.8)

    df_target['date'] = pd.to_datetime(df_target['date'])
    hourly_df['datetime'] = pd.to_datetime(hourly_df['datetime'])
    hourly_df.columns = hourly_df.columns.str.lower()
    
    if 'ATR_84' not in df_target.columns:
        df_target['ATR_84'] = ta.atr(df_target['ETH_High'], df_target['ETH_Low'], df_target['ETH_Close'], length=84)

    if 'ADX_14' not in df_target.columns:
        adx = ta.adx(df_target['ETH_High'], df_target['ETH_Low'], df_target['ETH_Close'], length=14)
        if adx is not None:
            df_target['ADX_14'] = adx.iloc[:, 0]
        else:
            df_target['ADX_14'] = 25.0 

    df_target = df_target.sort_values('date').reset_index(drop=True)
    hourly_df = hourly_df.sort_values('datetime').reset_index(drop=True)
    
    base_dates_ts = df_target['date'].astype(np.int64).values // 10**6
    base_atr = df_target['ATR_84'].fillna(method='ffill').fillna(0).to_numpy()
    base_adx = df_target['ADX_14'].fillna(25).to_numpy() 
    
    # 1시간봉 데이터 배열 준비
    hourly_dates_ts = hourly_df['datetime'].astype(np.int64).values // 10**6
    hourly_open = hourly_df['open'].astype(np.float64).to_numpy()
    hourly_high = hourly_df['high'].astype(np.float64).to_numpy()
    hourly_low = hourly_df['low'].astype(np.float64).to_numpy()

    # Numba 함수 호출 (인자 순서 정확히 맞춤)
    targets, entry_prices, tp_prices, sl_prices = compute_targets_with_hourly_numba_4h(
        base_dates_ts, base_atr, base_adx, 
        hourly_dates_ts, hourly_open, hourly_high, hourly_low,
        lookahead, profit_mult, stop_mult,
        use_dynamic_tp, adx_th_high, adx_th_low, boost_factor, penalty_factor 
    )
    
    df_target['next_direction'] = targets
    df_target['real_entry_price'] = entry_prices
    df_target['take_profit_price'] = tp_prices
    df_target['stop_loss_price'] = sl_prices
    
    df_target['next_close'] = df_target['ETH_Close'].shift(-1)
    df_target['next_open'] = df_target['ETH_Open'].shift(-1)
    df_target['next_log_return'] = np.log(df_target['next_close'] / (df_target['next_open'] + 1e-9))
    
    if lookahead > 0:
        df_target = df_target.iloc[:-lookahead]
    
    valid_before = len(df_target)
    df_target = df_target[df_target['next_direction'] != -1].reset_index(drop=True)
    valid_after = len(df_target)
    
    print(f"Valid Samples: {valid_after}/{valid_before} (Removed: {valid_before - valid_after})")
    
    return df_target




In [3]:
def select_features_verified(X_train, y_train, task='class', top_n=20, verbose=True):
    if len(X_train) > 10000:
        idx = np.random.choice(len(X_train), 10000, replace=False)
        X_sub = X_train.iloc[idx]
        y_sub = y_train.iloc[idx]
    else:
        X_sub, y_sub = X_train, y_train

    if task == 'class':
        mi_scores = mutual_info_classif(X_sub, y_sub, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_sub, y_sub, random_state=42, n_neighbors=3)
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    estimator = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1) if task == 'class' else LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    rfe = RFE(estimator=estimator, n_features_to_select=top_n, step=0.1, verbose=0)
    rfe.fit(X_sub, y_sub)
    rfe_features = X_train.columns[rfe.support_].tolist()

    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1) if task == 'class' else RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_sub, y_sub)
    rf_idx = np.argsort(rf_model.feature_importances_)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]
    
    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0: break
    
    return selected_features, {}

def select_features_multi_target(X_train, y_train, target_type='direction', top_n=20):
    must_have_cols = ['ATR_84', 'ADX_14']
    
    if target_type == 'direction':
        selected, stats = select_features_verified(X_train, y_train['next_direction'], task='class', top_n=top_n)
        
        for col in must_have_cols:
            if col in X_train.columns and col not in selected:
                if len(selected) >= top_n:
                    selected.pop()         
                selected.insert(0, col)  
            
    print(f"\n[Feature Selection] Top {len(selected)} Features Selected:")
    print(f" -> {', '.join(selected)}")
    return selected, stats

def process_single_split(split_data, target_type='direction', top_n=20, fold_idx=None, atr_col_name='ATR_84'): 
    train_df = split_data['train'] 
    val_df = split_data['val'] 
    test_df = split_data['test'] 
    fold_type = split_data.get('fold_type', 'unknown')

    print(f"\n{'='*80}")
    print(f" Processing Fold {fold_idx} ({fold_type})")
    print(f"{'='*80}")
    print(f" Train Period: {train_df['date'].min()} ~ {train_df['date'].max()} (N={len(train_df)})")
    print(f" Val   Period: {val_df['date'].min()} ~ {val_df['date'].max()} (N={len(val_df)})")
    print(f" Test  Period: {test_df['date'].min()} ~ {test_df['date'].max()} (N={len(test_df)})")

    target_cols = [
        'next_direction', 'next_log_return', 'next_close', 'next_open', 
        'take_profit_price', 'stop_loss_price', 
        'ATR_84', 'real_entry_price'
    ]
    if 'ADX_14' in train_df.columns:
        target_cols.append('ADX_14')

    train_processed = train_df.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_df.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_df.dropna(subset=target_cols).reset_index(drop=True)

    exclude_cols = [col for col in target_cols if col not in [atr_col_name, 'ADX_14']] + ['date']
    feature_cols = [col for col in train_processed.columns if col not in exclude_cols]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    balance = y_train['next_direction'].value_counts(normalize=True).to_dict()
    print(f"[Class Balance] Train Set: {balance}")

    selected_features, selection_stats = select_features_multi_target(
        X_train, y_train, target_type=target_type, top_n=top_n
    )

    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]

    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_sel), columns=selected_features)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val_sel), columns=selected_features)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_sel), columns=selected_features)

    return {
        'train': {'X': X_train_scaled, 'y': y_train.reset_index(drop=True), 'dates': train_processed['date'].reset_index(drop=True)},
        'val': {'X': X_val_scaled, 'y': y_val.reset_index(drop=True), 'dates': val_processed['date'].reset_index(drop=True)},
        'test': {'X': X_test_scaled, 'y': y_test.reset_index(drop=True), 'dates': test_processed['date'].reset_index(drop=True)},
        'scaler': scaler, 
        'selected_features': selected_features,
        'stats': {
            'fold_idx': fold_idx if fold_idx is not None else split_data.get('fold_idx', 0),
            'fold_type': split_data.get('fold_type', 'unknown')
        }
    }

def split_walk_forward_method(df, train_start_date, final_test_start='2025-01-01', 
                              initial_train_size=800, val_size=150, test_size=150, 
                              step=150, gap_size=30):
    
    df_period = df[df['date'] >= pd.to_datetime(train_start_date)].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    total_len = len(df_period)
    folds = []
    current_test_end = total_len
    
    while True:
        test_end_idx = current_test_end
        test_start_idx = test_end_idx - test_size
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0: break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_type': 'walk_forward_rolling_reverse'
        })
        current_test_end = test_start_idx - gap_size
    
    folds.reverse()
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        if len(pre_final_df) >= (initial_train_size + val_size + gap_size):
            final_val_end_idx = len(pre_final_df)
            final_val_start_idx = final_val_end_idx - val_size
            final_train_end_idx = final_val_start_idx - gap_size
            final_train_start_idx = final_train_end_idx - initial_train_size
            
            final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
            final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
            
            folds.append({
                'train': final_train_data,
                'val': final_val_data,
                'test': final_test_df,
                'fold_idx': len(folds) + 1,
                'fold_type': 'final_holdout'
            })
            
    return folds

def build_complete_pipeline_corrected(df_raw, df_hour, train_start_date, **kwargs): 
    print(f"\n Pipeline Started... (Train Start: {train_start_date})")


    lookahead = kwargs.get('lookahead_periods', 30) 
    profit_mult = kwargs.get('profit_mult', 2.0)
    stop_mult = kwargs.get('stop_mult', 1.0)
    use_dynamic_tp = kwargs.get('use_dynamic_tp', False)
    adx_threshold = kwargs.get('adx_threshold', 40)
    boost_factor = kwargs.get('boost_factor', 1.6)
    penalty_factor = kwargs.get('penalty_factor', 0.8) 
    df = create_targets_4h(df_raw, df_hour,
        lookahead=lookahead, 
        profit_mult=profit_mult, 
        stop_mult=stop_mult,
        use_dynamic_tp=use_dynamic_tp,
        adx_threshold=adx_threshold,
        boost_factor=boost_factor,
        penalty_factor=penalty_factor 
    )
    df = remove_raw_prices_and_transform(df, 'direction', 'tvt')
    print(f"Final Data Shape: {df.shape}")
    
    initial_train_size = kwargs.get('initial_train_days', 800) * 6
    val_size = 150 * 6
    test_size = 150 * 6
    gap_size = lookahead

    splits = split_walk_forward_method(
        df, 
        train_start_date=train_start_date,
        final_test_start=kwargs.get('final_test_start', '2025-01-01'),
        initial_train_size=initial_train_size,
        val_size=val_size,
        test_size=test_size,
        step=150 * 6,
        gap_size=gap_size
    )
    print(f"Data Split Completed. Total {len(splits)} folds generated.")

    result = []
    for fold in splits:
        res = process_single_split(
            fold, 
            top_n=kwargs.get('top_n', 20), 
            fold_idx=fold['fold_idx'],
            atr_col_name='ATR_84'
        )
        result.append(res)

    return result

In [4]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import gc
import traceback
import os
import joblib
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight


class DirectionModels:
    """
    내부 Optuna를 모두 제거하고, 외부에서 params를 주입받는 구조로 변경.
    """

    @staticmethod
    def random_forest(X_train, y_train, X_val, y_val, params=None):
        if params is None: params = {}
        
        # 강제 설정 (안전장치)
        safe_params = params.copy()
        safe_params.update({
            'n_jobs': -1,
            'random_state': 42,
            'class_weight': 'balanced'
        })
        
        model = RandomForestClassifier(**safe_params)
        model.fit(X_train, y_train)
        return model, safe_params

    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val, params=None):
        if params is None: params = {}
        
        safe_params = params.copy()
        safe_params.update({
            'objective': 'binary',
            'metric': 'binary_logloss',
            'class_weight': 'balanced',
            'verbosity': -1,
            'n_jobs': -1,
            'random_state': 42
        })

        model = lgb.LGBMClassifier(**safe_params)

        callbacks = [lgb.early_stopping(stopping_rounds=50, verbose=False)]
        
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
        return model, safe_params

    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val, params=None):
        # Class Weight 계산
        neg, pos = np.bincount(y_train.astype(int))
        scale_pos_weight = neg / pos if pos > 0 else 1.0

        if params is None: params = {}
        
        safe_params = params.copy()
        safe_params.update({
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'tree_method': 'hist',
            'scale_pos_weight': scale_pos_weight,
            'early_stopping_rounds': 50,
            'random_state': 42,
            'n_jobs': -1
        })

        model = xgb.XGBClassifier(**safe_params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        return model, safe_params

    @staticmethod
    def catboost(X_train, y_train, X_val, y_val, params=None):
        if params is None: params = {}

        safe_params = params.copy()
        safe_params.update({
            'loss_function': 'Logloss',
            'eval_metric': 'Logloss',
            'auto_class_weights': 'Balanced',
            'logging_level': 'Silent',
            'random_seed': 42,
            'od_type': 'Iter',
            'od_wait': 50,
            'allow_writing_files': False
        })

        model = cb.CatBoostClassifier(**safe_params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val))
        return model, safe_params

ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 2, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 3, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 4, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True}
]


In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.best_thresholds = {}
        self.save_models = save_models
        self.models = {} if save_models else None
        self.prediction_logs = {}
        self.ensemble_probs = {'train': [], 'val': [], 'test': []}
        self.y_trues = {}
        self.model_params = {}
        
    def _calculate_metrics(self, y_true, probs, threshold, rr_ratio, prefix=""):
        """
        공통 메트릭 계산 로직 (중복 제거)
        """
        preds = (probs >= threshold).astype(int)
        n_trades = np.sum(preds == 1)

        # 기본 ML 메트릭
        acc = accuracy_score(y_true, preds)
        prec = precision_score(y_true, preds, zero_division=0)
        rec = recall_score(y_true, preds, zero_division=0)
        f1 = f1_score(y_true, preds, zero_division=0)
        try: 
            auc = roc_auc_score(y_true, probs)
        except: 
            auc = 0.5

        # 트레이딩 메트릭 (Expectancy)
        if n_trades > 0:
            wins = np.sum((preds == 1) & (y_true == 1))
            losses = n_trades - wins
            win_rate = wins / n_trades
            # 기대 수익: (승리 * RR) - (패배 * 1)
            expectancy = ((wins * rr_ratio) - (losses * 1.0)) / n_trades
        else:
            win_rate, expectancy = 0.0, 0.0

        return {
            f"{prefix}Acc": acc, 
            f"{prefix}Prec": prec, 
            f"{prefix}Rec": rec,
            f"{prefix}F1": f1, 
            f"{prefix}AUC": auc,
            f"{prefix}Exp": expectancy, 
            f"{prefix}WinRate": win_rate, 
            f"{prefix}Trades": n_trades
        }

    def optimize_threshold_grid(self, y_true, buy_prob, min_trades=20, rr_ratio=1.5):

        best_score = -999.0
        best_th = 0.5
        
        # 50%부터 98%까지 1% 단위로 검사
        percentiles = np.arange(50, 99, 0.5)
        thresholds = np.percentile(buy_prob, percentiles)
        
        thresholds = np.unique(thresholds)
        
        for th in thresholds:
            if th < 0.5: continue
            
            preds = (buy_prob >= th).astype(int)
            n_trades = np.sum(preds == 1)
            
            if n_trades < min_trades: 
                continue
            
            wins = np.sum((preds == 1) & (y_true == 1))
            losses = n_trades - wins
            
            win_rate = wins / n_trades
            
            # 기대수익 계산
            exp = ((wins * rr_ratio) - (losses * 1.0)) / n_trades
            
            # [안전장치] 승률이 40% 미만이면 큰 페널티 (운 좋은 고수익 방지)
            if win_rate < 0.4: 
                exp -= 0.5
            
            # 점수: 기대수익 * log(거래횟수) -> 거래 횟수가 너무 적으면 점수 낮음
            score = exp * np.log1p(n_trades)
            
            if score > best_score:
                best_score = score
                best_th = th
                
        return best_th

    def evaluate_model(self, model, X_train, y_train, X_val, y_val, X_test, y_test, 
                       model_name, profit_mult=1.5, stop_mult=1.0,
                       test_dates=None, test_prices=None, y_test_df=None, model_params=None):  
        
        if model_params is not None:
            self.model_params[model_name] = model_params
        
        rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5
        
        # 확률 예측
        tr_prob = model.predict_proba(X_train)[:, 1]
        val_prob = model.predict_proba(X_val)[:, 1]
        te_prob = model.predict_proba(X_test)[:, 1]
        
        # 앙상블을 위해 확률 저장
        self.ensemble_probs['train'].append(tr_prob)
        self.ensemble_probs['val'].append(val_prob)
        self.ensemble_probs['test'].append(te_prob)
        
        # 정답지 저장 (앙상블용)
        if 'train' not in self.y_trues:
            self.y_trues['train'] = y_train
            self.y_trues['val'] = y_val
            self.y_trues['test'] = y_test
        
        # 최소 거래 횟수 설정 (검증 데이터의 2% 또는 20회 중 큰 값)
        min_trades_val = max(30, int(len(y_val) * 0.02))
        
        best_th = self.optimize_threshold_grid(
            y_val.astype(int), val_prob, 
            min_trades=min_trades_val, 
            rr_ratio=rr_ratio
        )
        self.best_thresholds[model_name] = best_th

        # 지표 계산
        metrics_train = self._calculate_metrics(y_train.astype(int), tr_prob, best_th, rr_ratio, "Train_")
        metrics_val = self._calculate_metrics(y_val.astype(int), val_prob, best_th, rr_ratio, "Val_")
        metrics_test = self._calculate_metrics(y_test.astype(int), te_prob, best_th, rr_ratio, "Test_")
        
        # 결과 통합
        result = {
            'Model': model_name,
            'Threshold': best_th,
            **metrics_train,
            **metrics_val,
            **metrics_test
        }
        
        print(f"    -> [EVAL] {model_name:<12} | Te_Exp: {result['Test_Exp']:.3f}R | Te_Trades: {result['Test_Trades']} | Th: {best_th:.4f}")

        # 예측 로그 저장 (백테스팅 상세 분석용)
        if test_dates is not None:
            self._save_prediction_log(model_name, test_dates, test_prices, te_prob, best_th, y_test, y_test_df)

        self.results.append(result)
        if self.save_models: 
            self.models[model_name] = model
        
        return result
    
    def _save_prediction_log(self, model_name, dates, prices, probs, threshold, y_true, y_test_df):
        """
        예측 로그 저장 로직 분리
        """
        # 가격 데이터 처리 (DataFrame vs Series/Array 대응)
        if isinstance(prices, pd.DataFrame):
            close_vals = prices['ETH_Close'].values
            open_vals = prices['ETH_Open'].values
            high_vals = prices['ETH_High'].values
            low_vals = prices['ETH_Low'].values
        else:
            close_vals = prices
            open_vals = high_vals = low_vals = np.zeros(len(probs))

        # TP/SL 데이터 처리
        if y_test_df is not None:
            tp_vals = y_test_df['take_profit_price'].values
            sl_vals = y_test_df['stop_loss_price'].values
            entry_vals = y_test_df['real_entry_price'].values
        else:
            tp_vals = sl_vals = entry_vals = np.zeros(len(probs))

        pred_df = pd.DataFrame({
            'timestamp': dates,
            'open': open_vals, 'high': high_vals, 'low': low_vals, 'close': close_vals,
            'prob': probs, 
            'threshold': threshold, 
            'signal': (probs >= threshold).astype(int),
            'actual_target': y_true.astype(int),
            'take_profit_price': tp_vals, 
            'stop_loss_price': sl_vals, 
            'real_entry_price': entry_vals
        })
        
        if not isinstance(pred_df.index, pd.DatetimeIndex):
            pred_df.set_index('timestamp', inplace=True)
            
        self.prediction_logs[model_name] = pred_df

    def evaluate_ensemble(self, profit_mult, stop_mult):
        if not self.ensemble_probs['test']: return None
        
        # Soft Voting (확률 평균)
        ens_val_prob = np.mean(self.ensemble_probs['val'], axis=0)
        ens_te_prob = np.mean(self.ensemble_probs['test'], axis=0)
        
        rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5
        min_trades_val = max(20, int(len(self.y_trues['val']) * 0.02))
        
        # 앙상블 모델에 대해서도 최적 임계값 다시 찾기
        best_th = self.optimize_threshold_grid(
            self.y_trues['val'].astype(int), ens_val_prob, 
            min_trades=min_trades_val, 
            rr_ratio=rr_ratio
        )
        
        # 테스트 데이터 평가
        metrics_test = self._calculate_metrics(
            self.y_trues['test'].astype(int), 
            ens_te_prob, 
            best_th, 
            rr_ratio, 
            "Test_"
        )        
        print(f"    -> [EVAL] ENSEMBLE     | Te_Exp: {metrics_test['Test_Exp']:.3f}R | Te_Trades: {metrics_test['Test_Trades']} | Th: {best_th:.4f}")
        
        ens_result = {
            'Model': 'Ensemble', 
            'Threshold': best_th,
            **metrics_test
        }
        self.results.append(ens_result)
        return ens_result

    def get_model_params(self): return self.model_params
    def get_summary_dataframe(self): return pd.DataFrame(self.results)
    def get_models_dict(self): return self.models or {}
    def get_prediction_logs(self): return self.prediction_logs
    def get_best_thresholds(self): return self.best_thresholds

class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    def _prepare_target(self, y_data):
        if isinstance(y_data, pd.DataFrame):
            y_data = y_data.iloc[:, 0].values
        elif isinstance(y_data, pd.Series):
            y_data = y_data.values
        y_data = np.array(y_data).flatten()
        y_data = np.nan_to_num(y_data, nan=0.0)
        return np.round(y_data).astype(int)

    def train_all_models(self, X_train, y_train, X_val, y_val, X_test, y_test, 
                         profit_mult, stop_mult, ml_models,
                         test_dates=None, test_prices=None):

        
        y_train_arr = self._prepare_target(y_train)
        y_val_arr = self._prepare_target(y_val)
        y_test_arr = self._prepare_target(y_test)
        
        real_rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5
        print(f"  [Info] Training with RR_Ratio: {real_rr_ratio:.2f}") 
        
        for config in ml_models:
            try:

                model_params = config.get('params', {})
                

                if config.get('needs_val', False):
                    model, best_params = config['func'](
                        X_train, y_train_arr, 
                        X_val, y_val_arr, 
                        params=model_params 
                    )
                else:
                    model, best_params = config['func'](
                        X_train, y_train_arr,
                        params=model_params 
                    )
                

                self.evaluator.evaluate_model(
                    model=model, 
                    X_train=X_train, 
                    y_train=y_train_arr, 
                    X_val=X_val, 
                    y_val=y_val_arr, 
                    X_test=X_test, 
                    y_test=y_test_arr,         
                    model_name=config['name'], 
                    profit_mult=profit_mult, 
                    stop_mult=stop_mult,
                    test_dates=test_dates, 
                    test_prices=test_prices,
                    y_test_df=y_test,
                    model_params=best_params
                )
                
                del model
                gc.collect()
                
            except Exception as e:
                print(f"[Error] Failed {config['name']}: {e}")
                traceback.print_exc()

                
def save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, result_dir, all_params):
    base_dir = f"{result_dir}/{trial_name}/fold_{fold_idx}_{fold_type}"
    os.makedirs(base_dir, exist_ok=True)
    
    print(f" Saving results to: {base_dir}")
    
    summary = evaluator.get_summary_dataframe()
    summary.to_csv(f"{base_dir}/fold_summary.csv", index=False)
    
    pred_logs = evaluator.get_prediction_logs()
    
    for model_name, df_log in pred_logs.items():
        df_log.to_csv(f"{base_dir}/predictions_{model_name}.csv")
        
        if 'actual_return' in df_log.columns:
            df_log['strategy_return'] = df_log['signal'] * df_log['actual_return']
            df_log['cumulative_return'] = (1 + df_log['strategy_return']).cumprod()
            df_log[['cumulative_return']].to_csv(f"{base_dir}/equity_curve_{model_name}.csv")

    for name, model in evaluator.get_models_dict().items():
        joblib.dump(model, f"{base_dir}/model_{name}.pkl")
            
    if 'scaler' in fold_data:
        joblib.dump(fold_data['scaler'], f"{base_dir}/scaler.pkl")
        
    feature_names = fold_data.get('selected_features', [])
    if isinstance(feature_names, pd.Index): feature_names = feature_names.tolist()
    
    meta_data = {
        'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'fold_idx': fold_idx,
        'model_thresholds': evaluator.get_best_thresholds(), 
        'input_features': feature_names,                   
        'scaling_needed': True if 'scaler' in fold_data else False,
        'optuna_parameters': all_params,
        'ml_hyperparameters': evaluator.get_model_params()
    }
    
    with open(f"{base_dir}/metadata.json", 'w') as f:
        json.dump(meta_data, f, indent=4)
            
    return summary

In [6]:
def run_optuna_optimization(df_merged, df_hour, ml_models, n_trials=30):
    
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d")
    RESULT_DIR = f"model_results/{TIMESTAMP}_ADX_ATR"
    os.makedirs(RESULT_DIR, exist_ok=True)
    
    LOG_PATH = f"{RESULT_DIR}/optuna_logs.csv"

    ohlc_cols = ['date', 'ETH_Open', 'ETH_High', 'ETH_Low', 'ETH_Close'] 
    
    lookup_df = df_merged[ohlc_cols].copy()
    lookup_df['date'] = pd.to_datetime(lookup_df['date'])
    lookup_df = lookup_df.set_index('date').sort_index()

    csv_columns = [
        'trial', 'lookahead', 'profit_mult', 'stop_mult', 'top_n', 'train_days',
        'use_dynamic_tp', 'boost_factor', 'penalty_factor', 'adx_threshold', 'score'
    ]
    
    existing_history = pd.DataFrame()
    if os.path.exists(LOG_PATH):
        try:
            existing_history = pd.read_csv(LOG_PATH)
            print(f"\n[Resume] Loaded {len(existing_history)} existing trials.")

            for col in csv_columns:
                if col not in existing_history.columns:
                    existing_history[col] = 0 # 혹은 NaN
        except: 
            pass

    if not os.path.exists(LOG_PATH):
        with open(LOG_PATH, "w") as f:
            f.write(",".join(csv_columns) + "\n")

    def objective(trial):
        nonlocal existing_history 
        
        use_dynamic_tp = trial.suggest_categorical('use_dynamic_tp', [True, False])
        boost = trial.suggest_float('boost_factor', 1.2, 1.8, step=0.2)       
        penalty = trial.suggest_float('penalty_factor', 0.7, 0.9, step=0.1)
        train_days = trial.suggest_int('train_days', 600, 900, step=50)
        s_mult = trial.suggest_float('stop_mult', 1.3, 1.9, step=0.1)
        p_mult = trial.suggest_float('profit_mult', 2.0, 4.5, step=0.25)
        lookahead = trial.suggest_int('lookahead', 6, 16, step=2)      
        top_n = trial.suggest_int('top_n', 20, 30, step=5) 
        adx_th = trial.suggest_int('adx_threshold', 20, 40, step=5)
        
        # (A) RandomForest
        rf_params = {
            'n_estimators': trial.suggest_int('rf_n_est', 200, 400),
            'max_depth': trial.suggest_int('rf_depth', 3, 6),       # 깊이 제한!
            'min_samples_leaf': trial.suggest_int('rf_leaf', 20, 100),
            'max_features': trial.suggest_categorical('rf_feat', ['sqrt', 'log2'])
        }
        
        # (B) LightGBM
        lgb_params = {
            'n_estimators': 500,
            'learning_rate': trial.suggest_float('lgb_lr', 0.01, 0.05),
            'num_leaves': trial.suggest_int('lgb_leaves', 15, 31),
            'max_depth': trial.suggest_int('lgb_depth', 3, 6),
            'reg_alpha': trial.suggest_float('lgb_l1', 0.1, 5.0),
            'reg_lambda': trial.suggest_float('lgb_l2', 0.1, 5.0),
            'subsample': trial.suggest_float('lgb_sub', 0.6, 0.9),
        }

        # (C) CatBoost
        cb_params = {
            'iterations': 500,
            'learning_rate': trial.suggest_float('cb_lr', 0.01, 0.05),
            'depth': trial.suggest_int('cb_depth', 4, 7),
            'l2_leaf_reg': trial.suggest_float('cb_l2', 3.0, 10.0),
            'bagging_temperature': trial.suggest_float('cb_bag', 0.0, 1.0)
        }
        
        # (D) XGBoost
        xgb_params = {
            'n_estimators': 500,
            'learning_rate': trial.suggest_float('xgb_lr', 0.01, 0.05),
            'max_depth': trial.suggest_int('xgb_depth', 3, 6),
            'min_child_weight': trial.suggest_int('xgb_mcw', 10, 50),
            'subsample': trial.suggest_float('xgb_sub', 0.6, 0.9),
            'reg_alpha': trial.suggest_float('xgb_l1', 0.1, 5.0),
            'reg_lambda': trial.suggest_float('xgb_l2', 0.1, 5.0),
        }

        # === [3] 모델 리스트 구성 (params 주입) ===

        current_ml_models = [
            {'name': 'CatBoost', 'func': DirectionModels.catboost, 'params': cb_params, 'needs_val': True},
            {'name': 'RandomForest', 'func': DirectionModels.random_forest, 'params': rf_params, 'needs_val': True},
            {'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'params': lgb_params, 'needs_val': True},
            {'name': 'XGBoost', 'func': DirectionModels.xgboost, 'params': xgb_params, 'needs_val': True}
        ]
        
      

        if not existing_history.empty:
            mask = (
                (existing_history['lookahead'] == lookahead) &
                (np.isclose(existing_history['profit_mult'], p_mult, atol=1e-5)) &
                (np.isclose(existing_history['stop_mult'], s_mult, atol=1e-5)) &
                (existing_history['top_n'] == top_n) &
                (existing_history['train_days'] == train_days) &
                (existing_history['use_dynamic_tp'] == use_dynamic_tp) &
                (np.isclose(existing_history['boost_factor'], boost, atol=1e-5)) &
                (np.isclose(existing_history['penalty_factor'], penalty, atol=1e-5)) &
                (existing_history['adx_threshold'] == adx_th) 
            )
            if mask.any():
                prev_score = existing_history.loc[mask, 'score'].values[0]
                print(f"[Skip] Already done. Score: {prev_score}")
                return prev_score

        dyn_str = "T" if use_dynamic_tp else "F"
        trial_name = (
        f"T{trial.number}_L{lookahead}_P{p_mult:.1f}_S{s_mult:.1f}_"
        f"N{top_n}_TD{train_days}_D{dyn_str}_B{boost:.1f}_Pen{penalty:.1f}_Adx_th{adx_th}" 
        )
        print(f"\n{'='*60}\n Starting {trial_name}\n{'='*60}")
        
        try:

            pipeline_result = build_complete_pipeline_corrected(
                df_raw=df_merged,
                df_hour=df_hour,
                train_start_date='2020-01-01',
                final_test_start='2025-01-01',
                lookahead_periods=lookahead,
                profit_mult=p_mult,
                stop_mult=s_mult,
                top_n=top_n,
                initial_train_days=train_days,
                use_dynamic_tp=use_dynamic_tp, 
                adx_threshold=adx_th, 
                boost_factor=boost,
                penalty_factor=penalty
            )
            
            fold_scores = []
            

            all_params = {
                'train_days': train_days, 'stop_mult': s_mult, 'profit_mult': p_mult,
                'lookahead': lookahead, 'top_n': top_n, 'use_dynamic_tp': use_dynamic_tp,
                'boost_factor': boost, 'penalty_factor': penalty, 'adx_th': adx_th,
                'rf_params': rf_params, 'lgb_params': lgb_params, 
                'xgb_params': xgb_params, 'cb_params': cb_params
            }
            
            for fold_data in pipeline_result:
                stats = fold_data.get('stats', {}) 
                fold_idx = stats.get('fold_idx', 0)
                fold_type = stats.get('fold_type', 'unknown')
                
                print(f"   >> Running Fold {fold_idx} ({fold_type})")
                
                # 메타데이터 저장을 위해 fold_data에 파라미터 주입
                fold_data.update({
                    'profit_mult': p_mult,
                    'stop_mult': s_mult,
                    'lookahead': lookahead,
                    'use_dynamic_tp': use_dynamic_tp,
                    'boost_factor': boost,
                    'penalty_factor': penalty,
                    'adx_th': adx_th
                })
                
                test_dates = pd.to_datetime(fold_data['test']['dates'])
                test_ohlc = lookup_df.reindex(test_dates).fillna(0)
                
                # 평가자/학습자 초기화
                evaluator = ModelEvaluator(save_models=True) 
                trainer = ModelTrainer(evaluator)
                
                # 학습 실행
                trainer.train_all_models(
                    fold_data['train']['X'], fold_data['train']['y'],
                    fold_data['val']['X'], fold_data['val']['y'],
                    fold_data['test']['X'], fold_data['test']['y'],
                    p_mult, s_mult, current_ml_models,
                    test_dates=test_dates, 
                    test_prices=test_ohlc
                )
                
                # 앙상블 평가
                ens_res = evaluator.evaluate_ensemble(p_mult, s_mult)

                summary = save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, RESULT_DIR, all_params)
                
                if ens_res:
                    fold_score = ens_res['Test_Exp']
                else:
                    fold_score = summary['Test_Exp'].max()
                    
                fold_scores.append(fold_score)
                
                del evaluator, trainer
                gc.collect()
            
            mean_score = np.mean(fold_scores) if fold_scores else -99.0
            std_score = np.std(fold_scores) + 1e-9 if fold_scores else 1.0
            final_score = mean_score / (std_score + 0.5)
            
            print(f"\n === Trial Score: {final_score:.4f} (Mean: {mean_score:.2f}R, Std: {std_score:.2f}) ===")
            
            # [수정 3] CSV 파일 쓰기 (새 파라미터 포함)
            with open(LOG_PATH, "a") as f:
                # csv_columns 순서대로 데이터 준비
                row_data = [
                    trial.number, lookahead, p_mult, s_mult, top_n, train_days,
                    use_dynamic_tp, boost, penalty, adx_th, final_score
                ]
                # 리스트 -> 문자열 변환
                row_str = ",".join(map(str, row_data))
                f.write(row_str + "\n")
                f.flush() 
                os.fsync(f.fileno())
            
            new_row_data = [
                trial.number, lookahead, p_mult, s_mult, top_n, train_days, 
                use_dynamic_tp, boost, penalty, adx_th, final_score
            ]
            new_row = pd.DataFrame([new_row_data], columns=csv_columns)
            
            if existing_history.empty: 
                existing_history = new_row
            else: 
                existing_history = pd.concat([existing_history, new_row], ignore_index=True)
                
            return final_score
            
        except Exception as e:
            print(f" [Error] Trial Failed: {e}")
            traceback.print_exc()
            return -99.0

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    
    print(f"\n[Optuna] Best Params: {study.best_params}")
    return study


In [ ]:
# -----------------------------------------------------------------------------
# 5. Main Execution Block
# -----------------------------------------------------------------------------
df_merged=pd.read_csv("eth_4hour_cal.csv")
df_hour = pd.read_csv("eth_hour.csv")
# [실제 실행]
study = run_optuna_optimization(df_merged, df_hour, ML_MODELS_CLASSIFICATION, n_trials=200)

print("==================================================")
print(f" Best Expectancy: {study.best_value:.4f}")
print("==================================================")



[I 2025-11-29 19:17:04,775] A new study created in memory with name: no-name-dcc76f1a-f4ff-4228-ba31-03311ef2b89d



[Resume] Loaded 0 existing trials.

 Starting T0_L10_P2.8_S1.6_N25_TD900_DF_B1.2_Pen0.7_Adx_th30

 Pipeline Started... (Train Start: 2020-01-01)
Valid Samples: 17816/17899 (Removed: 83)
Final Data Shape: (17816, 228)
Data Split Completed. Total 8 folds generated.

 Processing Fold 1 (walk_forward_rolling_reverse)
 Train Period: 2020-02-13 21:00:00 ~ 2022-08-01 17:00:00 (N=5400)
 Val   Period: 2022-08-03 13:00:00 ~ 2022-12-31 09:00:00 (N=900)
 Test  Period: 2023-01-02 05:00:00 ~ 2023-06-01 01:00:00 (N=900)
[Class Balance] Train Set: {0: 0.6864814814814815, 1: 0.31351851851851853}

[Feature Selection] Top 25 Features Selected:
 -> ADX_14, BB_WIDTH, curve-dex_eth_tvl_1d_chg, uniswap_eth_tvl_1d_chg, usdt_eth_mcap_pct_chg_24h, ATR_84, MACDH_72_156_54, PRICE_VS_LOW_30p, usdt_eth_mcap_ma180_ratio, aave_eth_tvl_ma180_ratio, DXY_ma180_ratio, VIX_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, makerdao_eth_tvl_ma180_ratio, SP500_ma180_ratio, GOLD_ma180_ratio, SMA_120_ma180_ratio, SMA_300_pct_chg_24


[Feature Selection] Top 25 Features Selected:
 -> ADX_14, usdt_eth_mcap_pct_chg_24h, BB_WIDTH, aave_eth_tvl_1d_chg, ETH_Bin_Low, ATR_84, PRICE_VS_LOW_30p, BREAKOUT_STR_120p, eth_chain_tvl_ma180_ratio, DXY_ma180_ratio, VIX_ma180_ratio, SP500_ma180_ratio, GOLD_ma180_ratio, SMA_120_ma180_ratio, SMA_300_pct_chg_24h, SMA_200_ma180_ratio, uniswap_eth_tvl_pct_chg_24h, uniswap_eth_tvl_1d_chg, aave_eth_tvl_pct_chg_24h, eth_chain_tvl_pct_chg_24h, optimism_tvl_1d_chg, makerdao_eth_tvl_1d_chg, makerdao_eth_tvl_pct_chg_24h, eth_chain_tvl_1d_chg, optimism_tvl_pct_chg_24h
   >> Running Fold 1 (walk_forward_rolling_reverse)
  [Info] Training with RR_Ratio: 1.72
    -> [EVAL] CatBoost     | Te_Exp: -0.045R | Te_Trades: 148 | Th: 0.5625
    -> [EVAL] RandomForest | Te_Exp: -0.235R | Te_Trades: 96 | Th: 0.5859
    -> [EVAL] LightGBM     | Te_Exp: 0.000R | Te_Trades: 0 | Th: 0.5122
    -> [EVAL] XGBoost      | Te_Exp: -0.412R | Te_Trades: 148 | Th: 0.6036
    -> [EVAL] ENSEMBLE     | Te_Exp: -0.047R | Te

[I 2025-11-29 19:20:01,784] Trial 0 finished with value: -0.19215845197704953 and parameters: {'use_dynamic_tp': False, 'boost_factor': 1.2, 'penalty_factor': 0.7, 'train_days': 900, 'stop_mult': 1.6, 'profit_mult': 2.75, 'lookahead': 10, 'top_n': 25, 'adx_threshold': 30, 'rf_n_est': 215, 'rf_depth': 4, 'rf_leaf': 50, 'rf_feat': 'sqrt', 'lgb_lr': 0.03899293621081384, 'lgb_leaves': 30, 'lgb_depth': 6, 'lgb_l1': 0.993759608287265, 'lgb_l2': 4.921523590301426, 'lgb_sub': 0.862421344206746, 'cb_lr': 0.04243626522887869, 'cb_depth': 7, 'cb_l2': 6.2208192967803, 'cb_bag': 0.41231955591457503, 'xgb_lr': 0.047102374004843545, 'xgb_depth': 3, 'xgb_mcw': 40, 'xgb_sub': 0.7553681511802872, 'xgb_l1': 0.6154826559311729, 'xgb_l2': 2.0356649946493324}. Best is trial 0 with value: -0.19215845197704953.



 === Trial Score: -0.1922 (Mean: -0.17R, Std: 0.40) ===

 Starting T1_L8_P3.5_S1.8_N30_TD600_DF_B1.2_Pen0.7_Adx_th40

 Pipeline Started... (Train Start: 2020-01-01)
Valid Samples: 17818/17901 (Removed: 83)
Final Data Shape: (17818, 228)
Data Split Completed. Total 10 folds generated.

 Processing Fold 1 (walk_forward_rolling_reverse)
 Train Period: 2020-02-14 05:00:00 ~ 2021-10-06 01:00:00 (N=3600)
 Val   Period: 2021-10-07 13:00:00 ~ 2022-03-06 09:00:00 (N=900)
 Test  Period: 2022-03-07 21:00:00 ~ 2022-08-04 17:00:00 (N=900)
[Class Balance] Train Set: {0: 0.6919444444444445, 1: 0.3080555555555556}

[Feature Selection] Top 30 Features Selected:
 -> ADX_14, usdt_eth_mcap_pct_chg_24h, VIX_ma180_ratio, SMA_120_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, GOLD_pct_chg_24h, BB_WIDTH, MACD_72_156_54, ATR_84, MACDH_72_156_54, PRICE_VS_LOW_30p, PRICE_VS_LOW_120p, PRICE_VS_LOW_360p, Corr_ETH_XRP_24h, makerdao_eth_tvl_ma180_ratio, SP500_ma180_ratio, uniswap_eth_tvl_ma180_ratio, GOLD_ma180_ratio,

...


 === Trial Score: -0.2547 (Mean: -0.20R, Std: 0.30) ===

 Starting T2_L16_P2.8_S1.8_N20_TD650_DT_B1.8_Pen0.9_Adx_th25

 Pipeline Started... (Train Start: 2020-01-01)
Valid Samples: 17810/17893 (Removed: 83)
Final Data Shape: (17810, 228)
Data Split Completed. Total 9 folds generated.

 Processing Fold 1 (walk_forward_rolling_reverse)
 Train Period: 2020-05-12 05:00:00 ~ 2022-02-21 01:00:00 (N=3900)
 Val   Period: 2022-02-23 21:00:00 ~ 2022-07-23 17:00:00 (N=900)
 Test  Period: 2022-07-26 13:00:00 ~ 2022-12-23 09:00:00 (N=900)
[Class Balance] Train Set: {0: 0.6202564102564102, 1: 0.37974358974358974}

[Feature Selection] Top 20 Features Selected:
 -> ADX_14, ATR_84, BB_WIDTH, MACDH_72_156_54, MFI_84, usdt_eth_mcap_ma180_ratio, DXY_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, makerdao_eth_tvl_ma180_ratio, uniswap_eth_tvl_ma180_ratio, GOLD_ma180_ratio, SMA_120_ma180_ratio, SMA_300_pct_chg_24h, SMA_200_ma180_ratio, uniswap_eth_tvl_pct_chg_24h, makerdao_eth_tvl_1d_chg, eth_chain_tvl_1d_chg,


[Feature Selection] Top 20 Features Selected:
 -> ADX_14, ATR_84, ETH_Bin_Close, PRICE_VS_LOW_30p, BREAKOUT_STR_120p, DXY_ma180_ratio, VIX_ma180_ratio, arbitrum_tvl_ma180_ratio, GOLD_ma180_ratio, usdt_eth_mcap_pct_chg_24h, uniswap_eth_tvl_pct_chg_24h, optimism_tvl_1d_chg, uniswap_eth_tvl_1d_chg, curve-dex_eth_tvl_1d_chg, eth_chain_tvl_pct_chg_24h, eth_chain_tvl_1d_chg, lido_eth_tvl_pct_chg_24h, lido_eth_tvl_1d_chg, curve-dex_eth_tvl_pct_chg_24h, arbitrum_tvl_pct_chg_24h
   >> Running Fold 1 (walk_forward_rolling_reverse)
  [Info] Training with RR_Ratio: 1.53
    -> [EVAL] CatBoost     | Te_Exp: -0.579R | Te_Trades: 90 | Th: 0.5109
    -> [EVAL] RandomForest | Te_Exp: -0.304R | Te_Trades: 98 | Th: 0.5859
    -> [EVAL] LightGBM     | Te_Exp: -0.171R | Te_Trades: 363 | Th: 0.5106
    -> [EVAL] XGBoost      | Te_Exp: 0.000R | Te_Trades: 0 | Th: 0.5135
    -> [EVAL] ENSEMBLE     | Te_Exp: -0.267R | Te_Trades: 100 | Th: 0.5241
 Saving results to: model_results/2025-11-29_ADX_ATR/T2_L16_P2.8

[I 2025-11-29 19:25:54,814] Trial 2 finished with value: -0.46003437376916295 and parameters: {'use_dynamic_tp': True, 'boost_factor': 1.8, 'penalty_factor': 0.8999999999999999, 'train_days': 650, 'stop_mult': 1.8, 'profit_mult': 2.75, 'lookahead': 16, 'top_n': 20, 'adx_threshold': 25, 'rf_n_est': 399, 'rf_depth': 4, 'rf_leaf': 76, 'rf_feat': 'log2', 'lgb_lr': 0.0435271444368459, 'lgb_leaves': 16, 'lgb_depth': 5, 'lgb_l1': 3.3643514581605123, 'lgb_l2': 3.6759823461924936, 'lgb_sub': 0.7592657885183483, 'cb_lr': 0.01888986987358045, 'cb_depth': 6, 'cb_l2': 4.517781773776159, 'cb_bag': 0.7116102090363221, 'xgb_lr': 0.03536631314162248, 'xgb_depth': 6, 'xgb_mcw': 30, 'xgb_sub': 0.6356020809280194, 'xgb_l1': 2.9372780679461683, 'xgb_l2': 4.27012729893932}. Best is trial 0 with value: -0.19215845197704953.



 === Trial Score: -0.4600 (Mean: -0.38R, Std: 0.32) ===

 Starting T3_L6_P2.2_S1.9_N20_TD750_DT_B1.8_Pen0.7_Adx_th20

 Pipeline Started... (Train Start: 2020-01-01)
Valid Samples: 17820/17903 (Removed: 83)
Final Data Shape: (17820, 228)
Data Split Completed. Total 9 folds generated.

 Processing Fold 1 (walk_forward_rolling_reverse)
 Train Period: 2020-02-18 21:00:00 ~ 2022-03-09 17:00:00 (N=4500)
 Val   Period: 2022-03-10 21:00:00 ~ 2022-08-07 17:00:00 (N=900)
 Test  Period: 2022-08-08 21:00:00 ~ 2023-01-05 17:00:00 (N=900)
[Class Balance] Train Set: {0: 0.7184444444444444, 1: 0.28155555555555556}

[Feature Selection] Top 20 Features Selected:
 -> ATR_84, usdt_eth_mcap_pct_chg_24h, MACDH_72_156_54, ADX_14, MFI_84, PRICE_VS_LOW_30p, PRICE_VS_LOW_120p, aave_eth_tvl_ma180_ratio, DXY_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, SP500_ma180_ratio, SMA_300_pct_chg_24h, eth_chain_tvl_pct_chg_24h, makerdao_eth_tvl_pct_chg_24h, curve-dex_eth_tvl_1d_chg, uniswap_eth_tvl_pct_chg_24h, curve-dex_e


[Feature Selection] Top 20 Features Selected:
 -> ADX_14, usdt_eth_mcap_pct_chg_24h, makerdao_eth_tvl_1d_chg, arbitrum_tvl_1d_chg, ATR_84, BREAKOUT_STR_120p, Price_div_VWMA_50d, Corr_ETH_XRP_24h, usdt_eth_mcap_ma180_ratio, DXY_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, aave_eth_tvl_1d_chg, makerdao_eth_tvl_pct_chg_24h, optimism_tvl_pct_chg_24h, uniswap_eth_tvl_1d_chg, uniswap_eth_tvl_pct_chg_24h, eth_chain_tvl_1d_chg, aave_eth_tvl_pct_chg_24h, eth_chain_tvl_pct_chg_24h, curve-dex_eth_tvl_1d_chg
   >> Running Fold 1 (walk_forward_rolling_reverse)
  [Info] Training with RR_Ratio: 1.18
    -> [EVAL] CatBoost     | Te_Exp: -0.410R | Te_Trades: 74 | Th: 0.5047
    -> [EVAL] RandomForest | Te_Exp: -0.688R | Te_Trades: 35 | Th: 0.5838
    -> [EVAL] LightGBM     | Te_Exp: -0.064R | Te_Trades: 14 | Th: 0.5071
    -> [EVAL] XGBoost      | Te_Exp: -0.587R | Te_Trades: 127 | Th: 0.5025
    -> [EVAL] ENSEMBLE     | Te_Exp: -0.584R | Te_Trades: 21 | Th: 0.5231
 Saving results to: model_results/202